In [0]:
import pyspark
from pyspark.sql import SparkSession
spark:SparkSession = SparkSession.builder\
      .master("local[1]")\
      .appName("Spark")\
      .getOrCreate()

In [0]:
columns = ["Seqno","Quote"]
data = [("1", "Be the change that you wish to see in the world"),
    ("2", "Everyone thinks of changing the world, but no one thinks of changing himself."),
    ("3", "The purpose of our lives is to be happy."),
    ("4", "Be cool.")]
df = spark.createDataFrame(data,columns)
df.show()

+-----+--------------------+
Seqno| Quote|
+-----+--------------------+
 1|Be the change tha...|
 2|Everyone thinks o...|
 3|The purpose of ou...|
 4| Be cool.|
+-----+--------------------+

In [0]:
#Display full column contents
df.show(truncate=False)

+-----+-----------------------------------------------------------------------------+
Seqno|Quote |
+-----+-----------------------------------------------------------------------------+
1 |Be the change that you wish to see in the world |
2 |Everyone thinks of changing the world, but no one thinks of changing himself.|
3 |The purpose of our lives is to be happy. |
4 |Be cool. |
+-----+-----------------------------------------------------------------------------+

In [0]:
# Display 2 rows and full column contents
df.show(2,truncate=False) 

+-----+-----------------------------------------------------------------------------+
Seqno|Quote |
+-----+-----------------------------------------------------------------------------+
1 |Be the change that you wish to see in the world |
2 |Everyone thinks of changing the world, but no one thinks of changing himself.|
+-----+-----------------------------------------------------------------------------+
only showing top 2 rows

In [0]:
# Display 2 rows & column values 25 characters
df.show(2,truncate=25)

+-----+-------------------------+
Seqno| Quote|
+-----+-------------------------+
 1|Be the change that you...|
 2|Everyone thinks of cha...|
+-----+-------------------------+
only showing top 2 rows

In [0]:
# Display DataFrame rows & columns vertically
df.show(n=3,truncate=25,vertical=True)

-RECORD 0--------------------------
 Seqno | 1 
 Quote | Be the change that you... 
-RECORD 1--------------------------
 Seqno | 2 
 Quote | Everyone thinks of cha... 
-RECORD 2--------------------------
 Seqno | 3 
 Quote | The purpose of our liv... 
only showing top 3 rows

In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

spark = SparkSession.builder.master("local[1]") \
                    .appName('Spark') \
                    .getOrCreate()

data = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show(truncate=False)

root
-- firstname: string (nullable = true)
-- middlename: string (nullable = true)
-- lastname: string (nullable = true)
-- id: string (nullable = true)
-- gender: string (nullable = true)
-- salary: integer (nullable = true)

+---------+----------+--------+-----+------+------+
firstname|middlename|lastname|id |gender|salary|
+---------+----------+--------+-----+------+------+
James | |Smith |36636|M |3000 |
Michael |Rose | |40288|M |4000 |
Robert | |Williams|42114|M |4000 |
Maria |Anne |Jones |39192|F |4000 |
Jen |Mary |Brown | |F |-1 |
+---------+----------+--------+-----+------+------+

In [0]:
structureData = [
    (("James","","Smith"),"36636","M",3100),
    (("Michael","Rose",""),"40288","M",4300),
    (("Robert","","Williams"),"42114","M",1400),
    (("Maria","Anne","Jones"),"39192","F",5500),
    (("Jen","Mary","Brown"),"","F",-1)
  ]
structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])

df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)

root
-- name: struct (nullable = true)
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
-- id: string (nullable = true)
-- gender: string (nullable = true)
-- salary: integer (nullable = true)

+--------------------+-----+------+------+
name |id |gender|salary|
+--------------------+-----+------+------+
{James, , Smith} |36636|M |3100 |
{Michael, Rose, } |40288|M |4300 |
{Robert, , Williams}|42114|M |1400 |
{Maria, Anne, Jones}|39192|F |5500 |
{Jen, Mary, Brown} | |F |-1 |
+--------------------+-----+------+------+

In [0]:
from pyspark.sql.functions import col,struct,when
updatedDF = df2.withColumn("OtherInfo", 
    struct(col("id").alias("identifier"),
    col("gender").alias("gender"),
    col("salary").alias("salary"),
    when(col("salary").cast(IntegerType()) < 2000,"Low")
      .when(col("salary").cast(IntegerType()) < 4000,"Medium")
      .otherwise("High").alias("Salary_Grade")
  )).drop("id","gender","salary")

updatedDF.printSchema()
updatedDF.show(truncate=False)

root
-- name: struct (nullable = true)
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
-- OtherInfo: struct (nullable = false)
 |-- identifier: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- Salary_Grade: string (nullable = false)

+--------------------+------------------------+
name |OtherInfo |
+--------------------+------------------------+
{James, , Smith} |{36636, M, 3100, Medium}|
{Michael, Rose, } |{40288, M, 4300, High} |
{Robert, , Williams}|{42114, M, 1400, Low} |
{Maria, Anne, Jones}|{39192, F, 5500, High} |
{Jen, Mary, Brown} |{, F, -1, Low} |
+--------------------+------------------------+